In [202]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from sklearn.preprocessing import LabelEncoder

# Define the path to the CSV file
data_file_path = "C:/Users/wdd45/OneDrive/바탕 화면/딥러닝응용/WISDM.csv"

# Load the data from the CSV file
data = pd.read_csv(data_file_path)

In [203]:
# Balance data for 'Sitting' and 'Jogging'
sitting_data = data[data['activity'] == 'Sitting'].head(4500).copy()
jogging_data = data[data['activity'] == 'Jogging'].head(4500).copy()

In [204]:
# Concatenate the balanced data
balanced_data = pd.concat([sitting_data, jogging_data])

In [205]:
# Encode the 'activity' column using LabelEncoder
label_encoder = LabelEncoder()
balanced_data['activity'] = label_encoder.fit_transform(balanced_data['activity'])

In [206]:
# Select only the specified columns
selected_columns = ['activity', 'X', 'Y', 'Z']
balanced_data = balanced_data[selected_columns]

# Display the resulting DataFrame
print(balanced_data)

        activity         X          Y         Z
221333         1  2.600000   9.660000  1.035146
221334         1  2.680000   9.530000  0.503953
221335         1  3.490000   8.890000  0.762740
221336         1  3.640000   9.380000  0.926184
221337         1  2.910000   9.340000  1.035146
...          ...       ...        ...       ...
13015          0  5.434519   8.853226  3.909040
13016          0 -4.331271  19.531578 -1.729784
13017          0 -1.035146  -5.012288 -4.481094
13018          0  7.082581  17.243360 -0.381370
13019          0  6.442424   5.053149 -5.788648

[9000 rows x 4 columns]


In [207]:
# Split the data into training and testing sets
X_train, X_test = train_test_split(balanced_data, test_size=0.25, random_state=42)


In [208]:
# Separate 'Normal' data for training
normal = X_train[X_train['activity'] == 0]
y_train = normal['activity']
X_train_normal_train = normal.drop(['activity'], axis=1)


In [209]:
# Separate target and features for testing
y_test = X_test['activity']
X_test = X_test.drop(['activity'], axis=1).values


In [210]:
# Model Training
input_dim = X_train_normal_train.shape[1]
encoding_dim = 14


In [211]:
input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="tanh", activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2), activation="relu")(encoder)
decoder = Dense(int(encoding_dim / 2), activation='tanh')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

nb_epoch = 100
batch_size = 32

In [212]:
autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])


In [213]:
checkpointer = ModelCheckpoint(filepath="model.h5", verbose=0, save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)

In [214]:


history = autoencoder.fit(X_train_normal_train, X_train_normal_train, epochs=nb_epoch, batch_size=batch_size, shuffle=True,
                          validation_data=(X_test, X_test), verbose=1, callbacks=[checkpointer, tensorboard]).history



Epoch 1/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
 96/106 [==========================>...] - ETA: 0s - loss: 52.3899 - accuracy: 0.7520WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x0000017200B4D3A8> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=1

106/106 [==============================] - 0s 4ms/step - loss: 7.8197 - accuracy: 0.9273 - val_loss: 7.5193 - val_accuracy: 0.9618
Epoch 45/100
106/106 [==============================] - 1s 5ms/step - loss: 7.7630 - accuracy: 0.9273 - val_loss: 7.6679 - val_accuracy: 0.9631
Epoch 46/100
106/106 [==============================] - 0s 3ms/step - loss: 7.7152 - accuracy: 0.9299 - val_loss: 7.6046 - val_accuracy: 0.9618
Epoch 47/100
106/106 [==============================] - 0s 3ms/step - loss: 7.6672 - accuracy: 0.9305 - val_loss: 7.5458 - val_accuracy: 0.9618
Epoch 48/100
106/106 [==============================] - 0s 3ms/step - loss: 7.6231 - accuracy: 0.9317 - val_loss: 7.5231 - val_accuracy: 0.9644
Epoch 49/100
106/106 [==============================] - 0s 4ms/step - loss: 7.5901 - accuracy: 0.9329 - val_loss: 7.5218 - val_accuracy: 0.9631
Epoch 50/100
106/106 [==============================] - 0s 4ms/step - loss: 7.5537 - accuracy: 0.9338 - val_loss: 7.3186 - val_accuracy: 0.9653
Epoch